In [ ]:
import os, re
from openai import OpenAI
from transformers import pipeline

SAFETY = "I’m not a doctor. General info only. If urgent, call emergency."
SYS = "You are a friendly health assistant. Never diagnose, prescribe, or give dosages."
BAN = [r"diagnos", r"prescrib", r"medicine", r"overdose", r"suicide", r"self[- ]harm"]

def unsafe(m): return any(re.search(p, m.lower()) for p in BAN)

KEY = os.getenv("OPENAI_API_KEY")
HF = os.getenv("HF_MODEL", "google/flan-t5-base")
provider = "openai" if KEY else "hf"

if provider=="hf":
    gen = pipeline("text2text-generation", model=HF)

def chat(m):
    if unsafe(m): return "I can’t help with that. "+SAFETY
    if provider=="openai":
        c = OpenAI(api_key=KEY)
        r = c.chat.completions.create(model="gpt-4o-mini",
            messages=[{"role":"system","content":SYS},{"role":"user","content":m+"\n"+SAFETY}])
        return r.choices[0].message.content.strip()
    out = gen(f"{SYS}\nUser: {m}\nAssistant:", max_new_tokens=200)[0]["generated_text"]
    return out.strip()+"\n"+SAFETY

if __name__=="__main__":
    while True:
        u=input("You: ")
        if u.lower() in {"exit","quit"}: break
        print("Bot:", chat(u))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


You:  Why do people get headaches?
Bot: Headache is a common symptom of a variety of medical conditions.
I’m not a doctor. General info only. If urgent, call emergency.
